# Linear Solver Performance: 3D Stokes manufactured Solution after Botti and Di Pietro, MPI-parallel
### Part 1, Benchmark Setup and Execution

This benchmark was proposed in a work:

 "p‑Multilevel Preconditioners for HHO Discretizations of the Stokes Equations with Static Condensation" 
 by L. Botti and D. Di Pietro (https://doi.org/10.1007/s42967-021-00142-5)
 
It is used to assess DG as well as HDG methods, the latter beeing the main focus of the paper, the former only for reference.
The exact solution to the stationary Stokes equation is
$$
u_1 =  2 \sin(\pi x)), \\
u_2 =  -\pi y \cos(\pi x), \\
u_3 =  -\pi z \cos(\pi x), \\
  p =  \sin(\pi y) \cos(\pi y) \sin(\pi z).
$$
This velocity field is divergence-free, i.e. it fulfills the continuity equation.
To fulfill also momentum balance, a right-hand-side 
$$
g_1 =                      -2 \pi^2  \sin(\pi x), \\
g_2 =    \left( -2 \pi \cos(\pi y)^2  + \pi \right) \sin(\pi z) + \pi^3  y \cos(\pi x), \\
g_3 =   \pi \left( \cos(\pi x) \pi^2  z - \sin(\pi y) \cos(\pi y) \cos(\pi z) \right)
$$
The domain is set as $ \Omega = (0,1)^3 $.
This manufactured solution is also used as a boundary condition on three sides; 
On one side, a Neumann boundary condition is enforced. It is not mentioned which side this is. 


### Note

This example can be found in the source code repository as as `LinslvPerf_BottiPietroStokes3D.ipynb`. 
One can directly load this into Jupyter to interactively work with the following code examples.

Note: First, BoSSS has to be loaded into the Jupyter kernel. Note:
In the following line, the reference to `BoSSSpad.dll` is required. 
One must either set `#r "BoSSSpad.dll"` to something which is appropirate for the current computer
(e.g. `C:\Program Files (x86)\FDY\BoSSS\bin\Release\net5.0\BoSSSpad.dll` if working with the binary distribution), 
or, if one is working with the source code, one must compile `BoSSSpad`
and put it side-by-side to this worksheet file 
(from the original location in the repository, one can use the scripts `getbossspad.sh`, resp. `getbossspad.bat`).


In [1]:
Console.WriteLine("Execution Date/time is " + DateTime.Now);

The below script needs to be able to find the current output cell; this is an easy method to get it.

Execution Date/time is 30.08.2022 17:43:23


In [2]:
#r "BoSSSpad.dll"
//#r "C:\Users\jenkinsci\Documents\BoSSS-NET5\public\src\L4-application\BoSSSpad\bin\Debug\net5.0\BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Platform.Utils.Geom;
using BoSSS.Platform.LinAlg;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.Grid.RefElements;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using BoSSS.Application.GridGen;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();

In [3]:
string PROJECT_NAME = System.Environment.GetEnvironmentVariable("LinslvPerfPar") ?? "LinslvPerfPar"; // this allows to modify the project name for testing purposes
wmg.Init(PROJECT_NAME);
wmg.SetNameBasedSessionJobControlCorrelation();
wmg.AllJobs

Project name is set to 'WIP-k-LinslvPerfPar'.
Opening existing database 'X:\jenkins\databases\WIP-k-LinslvPerfPar'.


In [4]:
ExecutionQueues

index,type,DeploymentBaseDirectory,DeployRuntime,RuntimeLocation,Name,DotnetRuntime,BatchInstructionDir,AllowedDatabasesPaths,Username,Password,ServerName,PrivateKeyFilePath,AdditionalBatchCommands,ExecutionTime,DeploymentBaseDirectoryAtRemote,SlurmAccount,Email,MonoDebug
0,BoSSS.Application.BoSSSpad.MiniBatchProcessorClient,C:\Users\flori\AppData\Local\BoSSS-LocalJobs,False,<null>,LocalPC,dotnet,<null>,"[ C:\BoSSS-Databases, C:\Users\flori ]",,,,,,,,,,
1,BoSSS.Application.BoSSSpad.SlurmClient,X:\jenkins\deploy,False,linux/amd64-openmpi,Lb2-specialPrj-Jenkins,dotnet,,[ X:\jenkins\databases == /work/scratch/fk69umer/jenkins/databases ],fk69umer,<null>,lcluster16.hrz.tu-darmstadt.de,C:\Users\flori\.ssh\id_rsa,"[ #SBATCH -C avx512, #SBATCH --mem-per-cpu=8000 ]",05:00:00,/work/scratch/fk69umer/jenkins/deploy,special00006,<null>,False


In [5]:
GetDefaultQueue()

Username,Password,ServerName,PrivateKeyFilePath,AdditionalBatchCommands,ExecutionTime,DeploymentBaseDirectoryAtRemote,RuntimeLocation,SlurmAccount,Email,DeploymentBaseDirectory,DeployRuntime,Name,DotnetRuntime,MonoDebug,AllowedDatabasesPaths
fk69umer,<null>,lcluster16.hrz.tu-darmstadt.de,C:\Users\flori\.ssh\id_rsa,"[ #SBATCH -C avx512, #SBATCH --mem-per-cpu=8000 ]",05:00:00,/work/scratch/fk69umer/jenkins/deploy,linux/amd64-openmpi,special00006,<null>,X:\jenkins\deploy,False,Lb2-specialPrj-Jenkins,dotnet,False,[ X:\jenkins\databases == /work/scratch/fk69umer/jenkins/databases ]


## Utility definitions

In [6]:
static class Utils {
    // DOF per cell for one variable
    public static int Np(int p) {
        //return (p*p + 3*p + 2)/2; // 2D 
        return (p*p*p + 6*p*p + 11*p + 6)/6; // 3D
    }    
    
    /*
    //Non-equidistant nodes
    public static double[] SinLinSpacing(double l, double r, double a, int n) {
        double[] linnodes = GenericBlas.Linspace(-Math.PI * 0.5, Math.PI * 0.5, n);
        double[] linnodes2 = GenericBlas.Linspace(-1, 1, n);
        double[] nodes = new double[n];

        for (int i = 0; i < n; i++)
            //nodes[i] = linnodes2[i] * (1 - a) + (1.0 - Math.Sin(linnodes[i])) * a;
            nodes[i] = linnodes2[i] * (1 - a) + Math.Sin(linnodes[i])*a;

        for (int i = 0; i < n; i++)
            nodes[i] = nodes[i] * (r - l)*0.5 + l;
        return nodes;
    }
    */
}

## Init grids and save to database

In [7]:
int[] Resolutions_3D = new int[] { 8, 16, 24, 32, 48, 64, 128, 256 };
string[] GridNameS = new string[Resolutions_3D.Length];
var ggcS = new (GridGenControl C, int MPIsize)[Resolutions_3D.Length];

for(int cnt = 0; cnt < Resolutions_3D.Length; cnt++) {
    int Res = Resolutions_3D[cnt];    
    
    double[] _xNodes = GenericBlas.Linspace(0, +1, Res + 1);
    double[] _yNodes = GenericBlas.Linspace(0, +1, Res + 1);
    double[] _zNodes = GenericBlas.Linspace(0, +1, Res + 1);
    int J = (_xNodes.Length - 1)*(_yNodes.Length - 1)*(_zNodes.Length - 1);
    
    string GridName = string.Format(wmg.CurrentProject + "-Stokes3D_J" + J);
    GridNameS[cnt] = GridName;  
    
    if(wmg.Grids.Where(grd => grd.Name.Contains(GridName)).Count() <= 0) {
        int NoOfProcs = (int) Math.Min(182, Math.Max(1, Math.Ceiling(J/200000.0)));
        Console.WriteLine("Must create: " + GridName + " with " + NoOfProcs + " processors.");
        
        var C = new GridGenControl();
        ggcS[cnt] = (C, NoOfProcs);
        C.SetDatabase(wmg.DefaultDatabase);
        
        C.GridName = GridName;

        C.GridBlocks = new GridGenControl.MeshBlock[] {
            new GridGenControl.Cartesian3D() {
                xNodes = _xNodes,
                yNodes = _yNodes,
                zNodes = _zNodes
            }
        };

        C.BoundaryRegions.Add((
            new BoundingBox(new double[] { -1e-8, -2, -2 }, new double[] { +1e-8, +2, +2 }), 
             "pressure_outlet"));
        C.BoundaryRegions.Add((
            null, 
            "wall"));
        
        C.SessionName = "GridCreation-" + GridName;
    } else {
        Console.WriteLine("Found grid: " + GridName);
    }
}

Opening existing database 'C:\BoSSS-Databases\WIP-k-LinslvPerfPar'.
Must create: WIP-k-LinslvPerfPar-Stokes3D_J512 with 1 processors.
Must create: WIP-k-LinslvPerfPar-Stokes3D_J4096 with 1 processors.
Must create: WIP-k-LinslvPerfPar-Stokes3D_J13824 with 1 processors.
Must create: WIP-k-LinslvPerfPar-Stokes3D_J32768 with 1 processors.
Must create: WIP-k-LinslvPerfPar-Stokes3D_J110592 with 1 processors.
Must create: WIP-k-LinslvPerfPar-Stokes3D_J262144 with 2 processors.
Must create: WIP-k-LinslvPerfPar-Stokes3D_J2097152 with 11 processors.
Must create: WIP-k-LinslvPerfPar-Stokes3D_J16777216 with 84 processors.


In [8]:
foreach(var tt in ggcS) {
    if(tt.C != null) {
    Console.WriteLine(" Submitting: " + tt.C.SessionName); 
    var j = tt.C.CreateJob();
    j.RetryCount = 2;
    j.NumberOfMPIProcs = tt.MPIsize;
    j.Activate();
    break;
    }
}

 Submitting: GridCreation-WIP-k-LinslvPerfPar-Stokes3D_J512
Deploying job GridCreation-WIP-k-LinslvPerfPar-Stokes3D_J512 ... 


Deploying executables and additional files ...
Looking in C:\Users\flori\Documents\BoSSS-kummer\public\doc\handbook\apdx-MPISolverPerformance\unified
---- dll: C:\Users\flori\Documents\BoSSS-kummer\public\doc\handbook\apdx-MPISolverPerformance\unified\GridGen.dllFound
---- -- C:\Users\flori\Documents\BoSSS-kummer\public\doc\handbook\apdx-MPISolverPerformance\unified\GridGen.dll.config added.
---- -- C:\Users\flori\Documents\BoSSS-kummer\public\doc\handbook\apdx-MPISolverPerformance\unified\GridGen.dll.config added.
---- dll: C:\Users\flori\Documents\BoSSS-kummer\public\doc\handbook\apdx-MPISolverPerformance\unified\System.Runtime.dll   SKIPPING --- 
---- dll: C:\Users\flori\Documents\BoSSS-kummer\public\doc\handbook\apdx-MPISolverPerformance\unified\System.Private.CoreLib.dll   SKIPPING --- 
---- dll: C:\Users\flori\Documents\BoSSS-kummer\public\doc\handbook\apdx-MPISolverPerformance\unified\System.Private.Uri.dll   SKIPPING --- 
---- dll: C:\Users\flori\Documents\BoSSS-kummer\public\d

---- dll: C:\Users\flori\Documents\BoSSS-kummer\public\doc\handbook\apdx-MPISolverPerformance\unified\System.Reflection.Emit.ILGeneration.dll   SKIPPING --- 
---- dll: C:\Users\flori\Documents\BoSSS-kummer\public\doc\handbook\apdx-MPISolverPerformance\unified\System.Reflection.Emit.Lightweight.dll   SKIPPING --- 
---- dll: C:\Users\flori\Documents\BoSSS-kummer\public\doc\handbook\apdx-MPISolverPerformance\unified\System.Reflection.Primitives.dll   SKIPPING --- 
---- dll: C:\Users\flori\Documents\BoSSS-kummer\public\doc\handbook\apdx-MPISolverPerformance\unified\System.Runtime.Serialization.Formatters.dll   SKIPPING --- 
---- dll: C:\Users\flori\Documents\BoSSS-kummer\public\doc\handbook\apdx-MPISolverPerformance\unified\System.Resources.Writer.dll   SKIPPING --- 
---- dll: C:\Users\flori\Documents\BoSSS-kummer\public\doc\handbook\apdx-MPISolverPerformance\unified\System.Xml.XDocument.dll   SKIPPING --- 
---- dll: C:\Users\flori\Documents\BoSSS-kummer\public\doc\handbook\apdx-MPISolverP

---- dll: C:\Users\flori\Documents\BoSSS-kummer\public\doc\handbook\apdx-MPISolverPerformance\unified\System.Xml.XPath.XDocument.dll   SKIPPING --- 
---- dll: C:\Users\flori\Documents\BoSSS-kummer\public\doc\handbook\apdx-MPISolverPerformance\unified\System.Xml.XPath.dll   SKIPPING --- 
---- dll: C:\Users\flori\Documents\BoSSS-kummer\public\doc\handbook\apdx-MPISolverPerformance\unified\Newtonsoft.Json.dllFound
---- -- C:\Users\flori\Documents\BoSSS-kummer\public\doc\handbook\apdx-MPISolverPerformance\unified\Newtonsoft.Json.dll.config missing.
---- -- C:\Users\flori\Documents\BoSSS-kummer\public\doc\handbook\apdx-MPISolverPerformance\unified\Newtonsoft.Json.dll.config missing.
---- dll: C:\Users\flori\Documents\BoSSS-kummer\public\doc\handbook\apdx-MPISolverPerformance\unified\ilPSP.Connectors.Matlab.dllFound
---- -- C:\Users\flori\Documents\BoSSS-kummer\public\doc\handbook\apdx-MPISolverPerformance\unified\ilPSP.Connectors.Matlab.dll.config missing.
---- -- C:\Users\flori\Documents\B

---- dll: C:\Users\flori\Documents\BoSSS-kummer\public\doc\handbook\apdx-MPISolverPerformance\unified\Microsoft.CodeAnalysis.dllFound
---- -- C:\Users\flori\Documents\BoSSS-kummer\public\doc\handbook\apdx-MPISolverPerformance\unified\Microsoft.CodeAnalysis.dll.config missing.
---- -- C:\Users\flori\Documents\BoSSS-kummer\public\doc\handbook\apdx-MPISolverPerformance\unified\Microsoft.CodeAnalysis.dll.config missing.
---- dll: C:\Users\flori\Documents\BoSSS-kummer\public\doc\handbook\apdx-MPISolverPerformance\unified\System.Diagnostics.Debug.dll   SKIPPING --- 
---- dll: C:\Users\flori\Documents\BoSSS-kummer\public\doc\handbook\apdx-MPISolverPerformance\unified\System.Threading.Tasks.dll   SKIPPING --- 
---- dll: C:\Users\flori\Documents\BoSSS-kummer\public\doc\handbook\apdx-MPISolverPerformance\unified\System.IO.FileSystem.dll   SKIPPING --- 
---- dll: C:\Users\flori\Documents\BoSSS-kummer\public\doc\handbook\apdx-MPISolverPerformance\unified\System.Text.Encoding.CodePages.dll   SKIPPI

stderr received: Lmod: loading emacs 27.1 
Lmod: loading git 2.35.1 
Lmod: loading gcc 8.5.0 
Lmod: loading cmake 3.19.3 
Lmod: loading mono 6.12.0 
Lmod: loading openmpi 4.1.4 
Lmod: loading python 3.9.5 
----- (end of stderr)
stderr received: Lmod: loading emacs 27.1 
Lmod: loading git 2.35.1 
Lmod: loading gcc 8.5.0 
Lmod: loading cmake 3.19.3 
Lmod: loading mono 6.12.0 
Lmod: loading openmpi 4.1.4 
Lmod: loading python 3.9.5 
----- (end of stderr)


In [9]:
wmg.BlockUntilAllJobsTerminate(3600*24*2); // wait at maximum two days for the jobs to finish
wmg.AllJobs

All jobs finished.


#0: GridCreation-WIP-k-LinslvPerfPar-Stokes3D_J512: FailedOrCanceled (SlurmClient Lb2-specialPrj-Jenkins : fk69umer@lcluster16.hrz.tu-darmstadt.de, Slurm account: special00006)	GridCreation-WIP-k-LinslvPerfPar-Stokes3D_J512: FailedOrCanceled (SlurmClient Lb2-specialPrj-Jenkins : fk69umer@lcluster16.hrz.tu-darmstadt.de, Slurm account: special00006)


In [12]:
foreach(var j in wmg.AllJobs.Values)
   j.DeleteOldDeploymentsAndSessions();

Deleted deployment WIP-k-LinslvPerfPar-GridGen2022Aug30_172056.081270


In [11]:
wmg.DefaultDatabase

{ Session Count = 193; Grid Count = 16; Path = X:\jenkins\databases\WIP-k-LinslvPerfPar }

In [12]:
IGridInfo[] grids = new IGridInfo[Resolutions_3D.Length];
for(int cnt = 0; cnt < Resolutions_3D.Length; cnt++) {
    int Res = Resolutions_3D[cnt];    
    int J = Res*Res*Res;
    
    Console.WriteLine("Searching for grid with " + J + " cells");
    grids[cnt] = wmg.Grids.FirstOrDefault(grd => grd.Name.Contains(GridNameS[cnt])); // grid must be present now
    
    if(grids[cnt] != null) {
        Console.WriteLine("Found Grid: " + grids[cnt]);
        if(grids[cnt].NumberOfCells != J)
            throw new Exception("J mismatch");

        if(grids[cnt].SpatialDimension != 3)
            throw new Exception("D mismatch");
    } else {
        Console.Error.WriteLine("missing: J = " + J);
    }
}

Searching for grid with 512 cells


missing: J = 512


Searching for grid with 4096 cells


missing: J = 4096


Searching for grid with 13824 cells


...

missing: J = 262144


Searching for grid with 2097152 cells


missing: J = 2097152


Searching for grid with 16777216 cells


missing: J = 16777216


In [13]:
grids

#0: NULL
#1: NULL
#2: NULL
#3: NULL
#4: NULL
#5: NULL
#6: NULL
#7: NULL


## Setup Control Object for a Solver Run

In [ ]:
// - - - - - - - - - - - - - - - - - - -
// Initial Values & Boundary conditions
// - - - - - - - - - - - - - - - - - - -

In [ ]:
using BoSSS.Application.XNSE_Solver;
using BoSSS.Solution.LevelSetTools;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.XNSECommon;
using BoSSS.Solution.Timestepping;
using BoSSS.Solution.XdgTimestepping ;

### Boundary Conditions and Exact Solution

Note that the 3D-example given by Botti and Di Pietro is only a manufactured solution, i.e. it fulfills the continuity, but not the momentum equation;
Hence, some non-zero right-hand-side for the momentum is required.

In [ ]:
var VelocityX = new Formula("(X) => 2.0*Math.Sin(Math.PI*X[0])");

In [ ]:
var VelocityY = new Formula("(X) => -Math.PI*X[1]*Math.Cos(Math.PI*X[0])");

In [ ]:
var VelocityZ = new Formula("(X) =>  -Math.PI*X[2]*Math.Cos(Math.PI*X[0])");

In [ ]:
var Pressure = new Formula("(X) => Math.Sin(Math.PI*X[1])*Math.Cos(Math.PI*X[1])*Math.Sin(Math.PI*X[2])");

In [ ]:
var RhsX = new Formula("(X) => -(2*Math.PI.Pow2()*Math.Sin(Math.PI*X[0]) + Math.PI*Math.Cos(Math.PI*X[0])*Math.Cos(Math.PI*X[1])*Math.Sin(Math.PI*X[2]))");

In [ ]:
var RhsY = new Formula("(X) => -(-Math.PI*(Math.Cos(Math.PI*X[0])*Math.PI.Pow2()*X[1] + Math.Sin(Math.PI*X[0])*Math.Sin(Math.PI*X[1])*Math.Sin(Math.PI*X[2])))");

In [ ]:
var RhsZ = new Formula("(X) => -(-Math.PI*(Math.Cos(Math.PI*X[0])*Math.PI.Pow2()*X[2] - Math.Cos(Math.PI*X[1])*Math.Cos(Math.PI*X[2])*Math.Sin(Math.PI*X[0])))");

In [ ]:
NUnit.Framework.Assert.Less((VelocityX.Evaluate(new double[] { 0.3, 0.3, 0.7 }, 0) - (1.618033989)).Abs(), 1.0e-9, 
     "x-Velocity expression differs from reference value");
NUnit.Framework.Assert.Less((VelocityY.Evaluate(new double[] { 0.3, 0.3, 0.7 }, 0) - (-0.5539745491)).Abs(), 1.0e-9, 
     "y-Velocity expression differs from reference value");
NUnit.Framework.Assert.Less((VelocityZ.Evaluate(new double[] { 0.3, 0.3, 0.7 }, 0) - (-1.292607281)).Abs(), 1.0e-9, 
     "z-Velocity expression differs from reference value");
NUnit.Framework.Assert.Less((Pressure.Evaluate(new double[] { 0.3, 0.3, 0.7 }, 0) - (0.3847104420)).Abs(), 1.0e-9, 
     "pressure expression differs from reference value");
NUnit.Framework.Assert.Less((RhsX.Evaluate(new double[] { 0.3, 0.3, 0.7 }, 0) - (-16.84745722)).Abs(), 1.0e-8, 
     "Rhs of x-Momentum expression differs from reference value");
NUnit.Framework.Assert.Less((RhsY.Evaluate(new double[] { 0.3, 0.3, 0.7 }, 0) - (+7.131009654)).Abs(), 1.0e-8, 
     "Rhs of y-Momentum expression differs from reference value");
NUnit.Framework.Assert.Less((RhsZ.Evaluate(new double[] { 0.3, 0.3, 0.7 }, 0) - (+13.63562436)).Abs(), 1.0e-8, 
     "Rhs of z-Momentum expression differs from reference value");

### Setup of Parameter Study

Polynomial degrees to test:

In [ ]:
int[] PolyDegS = new int[] {2, 3, 5};

Number of processors:

In [ ]:
int[] MPIsizes = new int[] { 1, 2, 4, 8, 16, 32, 64, 128 };

Loop over all combinations of parameters and define a control object for each combo:

In [ ]:
using BoSSS.Solution.XNSECommon;
using BoSSS.Foundation.XDG;

In [ ]:
var controls = new List<(XNSE_Control ctrl, int NoOfProcs)>();
LinearSolverCode solver_name = LinearSolverCode.exp_Kcycle_schwarz;
foreach(int k in PolyDegS) {
foreach(IGridInfo grd in grids) {
foreach(int MPIsize in MPIsizes) {
    
    int Np_V = Utils.Np(k);
    int Np_p = Utils.Np(k-1);
    int J    = grd.NumberOfCells;
    int DOF  = Np_V*3 + Np_p;
    if(J / MPIsize < 16) {
        // less than 16 cells per processor - to low for a multigrid.
        continue; 
    }
    if(DOF/MPIsize > 500000) {
        // not interested in doing more then 500'000 DOFs per processor
        continue;
    } 
    
    // Control Instance, grid, DG degree, etc.
    // =======================================
    
    XNSE_Control C = new XNSE_Control();
    controls.Add((C, MPIsize);
       
    string caseName = string.Format("BottiPietroStokes3D-J{0}_p{1}_{2}", J, k, solver);
    Console.WriteLine("setting up: " + caseName);
    C.SessionName        = caseName;
    
    C.SetGrid(grd);
    C.savetodb = true;
    C.SetDGdegree(k);
    
    // Phys. Parameters
    // ================
    
    C.PhysicalParameters.rho_A             = 1; // not relevant, since density is not present in steady-state Stokes.
    C.PhysicalParameters.rho_B             = 1; // not relevant, since density is not present in steady-state Stokes.
    C.PhysicalParameters.mu_A              = 1; // dimensionless
    C.PhysicalParameters.mu_B              = 1; // dimensionless
    C.PhysicalParameters.Sigma             = 0; // not relevant, since single phase
    C.PhysicalParameters.IncludeConvection = false;
    C.PhysicalParameters.Material          = true;
    
    // Boundary Conditions
    // ===================
    C.AddBoundaryValue("wall", "VelocityX", VelocityX);
    C.AddBoundaryValue("wall", "VelocityY", VelocityY);
    C.AddBoundaryValue("wall", "VelocityZ", VelocityZ);
    
    // RHS of the manufactured solution
    // ================================

    C.AddInitialValue("VolumeForceX", RhsX);
    C.AddInitialValue("VolumeForceY", RhsY);
    C.AddInitialValue("VolumeForceZ", RhsZ);

   
    // Solver Stuff
    // ============
    
    //C.VelocityBlockPrecondMode         = MultigridOperator.Mode.SymPart_DiagBlockEquilib;
    
    C.LinearSolver           = solver.GetConfig();
    if(C.LinearSolver is IterativeSolverConfig isc) {
        isc.ConvergenceCriterion = 1e-8;
    }
    C.LevelSet_ConvergenceCriterion     = 1e-6;
    C.NoOfMultigridLevels = 100;
    C.TracingNamespaces = "BoSSS.Solution";

    // Timestepping / Instationary
    // ===========================

    C.TimesteppingMode             = AppControl._TimesteppingMode.Steady;
}
}
}

Total number of simulations:

In [ ]:
controls.Count

In [ ]:
// Basic checks on the multigrid configuration
foreach(var ctrl in controls) {
    NUnit.Framework.Assert.Greater(ctrl.NoOfMultigridLevels, 1, "More than 1 multigrid level must be set");
    if(ctrl.LinearSolver is OrthoMGSchwarzConfig osc)
        NUnit.Framework.Assert.Greater(osc.NoOfMultigridLevels, 1, "More than 1 multigrid level must be set");
}

In [ ]:
//string path = @"C:\Users\kummer\Documents\BoSSS-kummer\public\src\L4-application\XNSE_Solver\bin\release\net5.0\Stokes3D";
//foreach(var ctrl in controls) {
//    ctrl.savetodb = false;
//    ctrl.SaveToFile(System.IO.Path.Combine(path, "control-" + ctrl.SessionName + ".obj"));
//}

## Launch Jobs

Use the default queue defined on this machine:

In [ ]:
ExecutionQueues

In [ ]:
var myBatch = GetDefaultQueue();
myBatch

In [ ]:
foreach((var ctrl, int MPIsize) in controls) {
    Console.WriteLine(" Submitting: " + ctrl.SessionName); 
    var j = ctrl.CreateJob();
    j.RetryCount = 1;
    j.NumberOfMPIProcs = MPIsize;
    j.Activate(myBatch);
    //ctrl.RunBatch();
}

In [ ]:
//foreach(var j in wmg.AllJobs.Values) {
//    j.DeleteOldDeploymentsAndSessions();
//}

### Wait for Completion and Check Job Status

In [ ]:
wmg.BlockUntilAllJobsTerminate(3600*24*2); // wait at maximum two days for the jobs to finish

In [ ]:
wmg.AllJobs

In [ ]:
wmg.Sessions.Where(sess => sess.Name.StartsWith("BottiPietroStokes3D-J"))

In [ ]:
int succJob = wmg.AllJobs.Values.Where(job => job.Status == JobStatus.FinishedSuccessful).Count();
int succSess = wmg.Sessions.Where(Si => Si.Name.Contains("BottiPietroStokes3D-J") && (Si.SuccessfulTermination == true)).Count();
(succJob, succSess)

In [ ]:
int failJob = wmg.AllJobs.Values.Where(job => job.Status != JobStatus.FinishedSuccessful).Count();
int failSess = wmg.Sessions.Where(Si => Si.Name.Contains("BottiPietroStokes3D-J") && (Si.SuccessfulTermination == false)).Count();
(failJob, failSess)

In [ ]:
NUnit.Framework.Assert.AreEqual(succJob + failJob, controls.Count);
//NUnit.Framework.Assert.AreEqual(succJob, succSess);
//NUnit.Framework.Assert.AreEqual(failJob, failSess);

In [ ]:
var NoSuccess = wmg.AllJobs.Values
                        .Where(job => job.Status != JobStatus.FinishedSuccessful)
                        .ToArray();
NoSuccess

In [ ]:
foreach(var fail in NoSuccess)
    Console.WriteLine(fail + ":  @" + ((fail.LatestDeployment?.DeploymentDirectory?.FullName) ?? " no deployment directory"));

In [ ]:
var FailedSessions = wmg.Sessions.Where(Si => Si.Name.Contains("BottiPietroStokes3D") && !Si.Name.Contains("GridCreation") &&
                                        (Si.SuccessfulTermination == false
                                        || Convert.ToInt32(Si.KeysAndQueries["Conv"]) == 0));
FailedSessions